In [134]:
import gzip
import scipy
import scipy.optimize
import random
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from statistics import median
import random
import sklearn
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from urllib.request import urlopen
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [135]:
def readJSON(path):
    for l in gzip.open(path, 'rt', encoding='UTF-8'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u, g, d

## Question 1 (Baseline)

In [136]:
#Splitting into training and validation sets
raw_data = []
for user, game, d in readJSON("train.json.gz"):
    raw_data.append([user, game, d])

random.shuffle(raw_data)    
    
raw_train = raw_data[:175000]
raw_valid = raw_data[165000:]

In [137]:
#Creating a dict with user as key and all games played by him as values
gamesOfUser = defaultdict(set)
for item in raw_data:
    gamesOfUser[item[0]].add(item[1])

#Creating a set with all the games
allGames = set()
# allGamesList = sorted(list(allGames))
# allGamesDict = {}
# for i, g in enumerate(allGamesList):
#     allGamesDict[g] = i/len(allGamesList)
    
for item in raw_data:
    allGames.add(item[1])

In [138]:
trainPos = []
for item in raw_train:
    tup = (item[0], item[1], 1)
    trainPos.append(tup)
    
trainNeg = []
for z in range(1):
    for item in raw_train:
        user = item[0]
        games = gamesOfUser[user]
        notPlayedGames = allGames - games
        randomOne = random.sample(notPlayedGames, 1)[0]
        tup = (user, randomOne, 0)
        trainNeg.append(tup)

len(trainPos), len(trainNeg)

(175000, 175000)

In [139]:
#Modifying validation set
valid = []
for item in raw_valid:
    tup = (item[0], item[1], 1)
    valid.append(tup)

#Creating a negative validation elements and appending to validation set (valid)
for item in raw_valid:
    user = item[0]
    games = gamesOfUser[user]
    notPlayedGames = allGames - games
    randomOne = random.sample(notPlayedGames, 1)[0]
    tup = (user, randomOne, 0)
    valid.append(tup)    

In [140]:
### Would-play baseline: just rank which games are popular and which are not, and return '1' if a game is among the top-ranked

gameCount = defaultdict(int)
totalPlayed = 0

for user, game, _ in raw_train:
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort(reverse = True)

maxi = 0
for item in mostPopular:
    maxi = max(maxi, item[0])

mostPopularDict = defaultdict(float)
for item in mostPopular:
    mostPopularDict[item[1]] = item[0]/maxi

In [141]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [142]:
#Creating 2 dicts. One with game and one with user

gamesOfUser = defaultdict(set)
usersOfGame = defaultdict(set)

#Only on training set
for item in raw_train:
    gamesOfUser[item[0]].add(item[1])
    usersOfGame[item[1]].add(item[0])

In [143]:
def mostSimilar(u, g):
    similarities1 = []
    users = usersOfGame[g]
    for g2 in gamesOfUser[u]:
        if g == g2: continue # other than the query
        sim = Jaccard(users, usersOfGame[g2])
        similarities1.append(sim)
    
#     similarities2 = []
#     games = gamesOfUser[u]
#     for u2 in usersOfGame[g]:
#         if u == u2: continue # other than the query
#         sim = Jaccard(games, gamesOfUser[u2])
#         similarities2.append(sim)
    
#     count = 0
#     #Thershold set as 0.008
#     threshold = 0.008
#     for sim in similarities:
#         if sim > threshold:
#             count += 1
    
    #Since it is given most of the values should cross. Assuming it to be more than 50%.
    #Can be done using median too
#     if count > 0.5*len(similarities):
#         return 1

#     if len(similarities1) > 0:
#         min1 = min(similarities1)
#     else:
#         min1 = 0.002459177358905974
    
    
    
#     max1 = max(similarities1) if len(similarities1) > 0 else 0.036477232467427144
    mean1 = sum(similarities1)/len(similarities1) if len(similarities1) > 0 else 0.013651729456640182
    #median1 = median(similarities1) if len(similarities1) > 0 else 0
    
#     min2 = min(similarities2) if len(similarities2) > 0 else 0.008970185399297994
#     max2 = max(similarities2) if len(similarities2) > 0 else 0.0960382978251585
#     mean2 = sum(similarities2)/len(similarities2) if len(similarities2) > 0 else 0.03420200386502156
    #median2 = median(similarities2) if len(similarities2) > 0 else 0
    
#     pop = mostPopularDict[g]
#     if pop == 0:
#         pop = 0.12058540079119132
    
#     senti = [0, 0, 0, 0]
#     if g in sentiDict:
#         senti = sentiDict[g]
    
    return mean1 #, min2, max2, mean2+ senti

In [144]:
trainX = []
trainY = []
for tup in trainPos:
    trainX.append(mostSimilar(tup[0], tup[1]))
    trainY.append(tup[2])
    
for tup in trainNeg:
    trainX.append(mostSimilar(tup[0], tup[1]))
    trainY.append(tup[2])

In [145]:
validX = []
for item in valid:
    validX.append(mostSimilar(item[0], item[1]))

In [146]:
predictions = []
for i in validX:
    if i > 0.009852:
        predictions.append(1)
    else:
        predictions.append(0)
        
actualValues = []
for item in valid:
    actualValues.append(item[2])    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

Validation Accuracy: 86.19


In [148]:
f = open('pairs_Played.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Played_Out.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    tmp = mostSimilar(u, g)
    res = 0
    if tmp > 0.00986:
        res = 1
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

## Question 4 (Both Jaccard and Popularity)

In [ ]:
out = 0

predictions = []
for item in valid:
    #Only checking Jaccard if it is popular in the first place
    if item[1] in return1:
        predictions.append(mostSimilar(item[0], item[1]))
    else:
        predictions.append(0)
    
actualValues = []
for item in valid:
    actualValues.append(item[2])    

correctValues = [a == b for a, b in zip(predictions, actualValues)]

print('Validation Accuracy: ' + str(sum(correctValues)/len(valid)*100))

## Question 5 (Test Set and Submitting to Kaggle)

In [ ]:
f = open('pairs_Played.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Played_Final.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    if g in return1:
        res = mostSimilar(u, g)
    else:
        res = 0
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

In [ ]:
print('Submitted to Kaggle')
print('\nKaggle Username: aditya1c')

## Question 9

In [13]:
import gzip
import scipy
import scipy.optimize
import random
import numpy as np
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [14]:
def readJSON1(path):
    for l in gzip.open(path, 'rt', encoding='UTF-8'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u, g, d

In [15]:
#Splitting into training and validation sets
dataset = []
for user, game, d in readJSON1("train.json.gz"):
    dataset.append([user, game, d['hours_transformed']])
    
train = dataset[:165000]
valid = dataset[165000:]

In [ ]:
hoursPerUser = defaultdict(list) 
hoursPerGame = defaultdict(list)
allHours = []

for user, game, h in train:
    allHours.append(h)
    #These variables are not used much. Just to maintain consistency with workbook code
    hoursPerUser[user].append(h)
    hoursPerGame[game].append(h)

In [ ]:
N = len(dataset)
nUsers = len(hoursPerUser)
nGames = len(hoursPerGame)
users = list(hoursPerUser.keys())
games = list(hoursPerGame.keys())

In [ ]:
alpha = sum(allHours)/len(allHours)
userBiases = defaultdict(float)
gameBiases = defaultdict(float)

Defining Functions which are similar to those discussed in the class

In [ ]:
def prediction(user, game):
    return alpha + userBiases[user] + gameBiases[game]

In [ ]:
def unpack(theta):
    global alpha
    global userBiases
    global gameBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    gameBiases = dict(zip(games, theta[1+nUsers:]))

In [ ]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, g) for u, g, h in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in gameBiases:
        cost += lamb*gameBiases[i]**2
    return cost

In [ ]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dGameBiases = defaultdict(float)
    for u, g, h in dataset:
        u,i = u, g
        pred = prediction(u, i)
        diff = pred - h
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dGameBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in gameBiases:
        dGameBiases[i] += 2*lamb*gameBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dGameBiases[i] for i in games]
    return np.array(dtheta)

In [ ]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

Training on training set with lambda = 1

In [ ]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nGames),
                             derivative, args = (allHours, 1))

Testing on Validation Set

In [ ]:
allHoursValid = []
for u, g, h in valid:
    allHoursValid.append(h)

In [ ]:
#This is to use if user or game are not found in test set
ratingMean = sum(allHoursValid)/len(allHoursValid)
ratingMean

In [ ]:
predictionsValid = []
for u, g, h in valid:
    predictionsValid.append(prediction(u, g))

In [ ]:
mseValid = MSE(predictionsValid, allHoursValid)
print('MSE on Validation Set with lambda as 1 is ' + str(mseValid))

## Question 10

In [ ]:
#Sorting using lambda and key
sorted_users = sorted(userBiases.items(), key = lambda x: x[1])
smallestUser = sorted_users[0]
largestUser = sorted_users[-1]

In [ ]:
sorted_games = sorted(gameBiases.items(), key = lambda x: x[1])
smallestGame = sorted_games[0]
largestGame = sorted_games[-1]

In [ ]:
print('Smallest User Bias: ' + str(smallestUser))
print('\nLargest User Bias: ' + str(largestUser))

In [ ]:
print('Smallest Game Bias: ' + str(smallestGame))
print('\nLargest Game Bias: ' + str(largestGame))

## Question 11

Testing for lambda from 0 to 1 in steps of 0.05

In [ ]:
bestLambda = 0
bestMSE = float('inf')
lambdaList = []
mseList = []

lamb = 0
step = 0.05
#Training and checking using Lambda from 0 to 1
for i in range(int(1/step) + 1):
    tmpLamb = lamb + 0.05*i
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nGames),
                             derivative, args = (allHours, tmpLamb), disp = False)
    predictionsValid = []
    for u, g, h in valid:
        predictionsValid.append(prediction(u, g))
    mseValid = MSE(predictionsValid, allHoursValid)
    lambdaList.append(tmpLamb)
    mseList.append(mseValid)
    if mseValid < bestMSE:
        bestMSE = mseValid
        bestLambda = tmpLamb

The above code block output is run locally but not showed it here as the output is too long

In [ ]:
print('Best Lambda: ' + str(bestLambda))
print('Best MSE: ' + str(bestMSE))

In [ ]:
plt.plot(lambdaList, mseList)
plt.xlabel('Lambda')
plt.ylabel('MSE')
plt.title('Variation of MSE with Lambda')

Best Model is used to predict the test set and uploaded to Kaggle

In [ ]:
f = open('pairs_Hours.txt', 'r')
lines = f.readlines()
f.close()

f = open('pairs_Hours_Final.txt', 'w')
f.write('userID-gameID,prediction\n')
lines = lines[1:]
for line in lines:
    u, g = line.strip().split('-')
    res = prediction(u, g)
    #This can be replaced with 0 if required too. Just used for checking in Kaggle
    if u not in userBiases or g not in gameBiases:
        res = ratingMean
        
    f.write(u + '-' + g + ',' + str(res) + '\n')

f.close()

In [ ]:
print('Submitted to Kaggle')
print('\nKaggle Username: aditya1c')